In [1]:
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (confusion_matrix,precision_score,recall_score,f1_score,
    roc_curve,roc_auc_score,precision_recall_curve,accuracy_score,classification_report)
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans,AgglomerativeClustering,DBSCAN
from sklearn.metrics import silhouette_score
import time

We use a dataset containing movie reviews from the *Internet Movie Database*. For this, the data first needs to be downloaded from <a href="http://ai.stanford.edu/~amaas/data/sentiment/"> here </a>. Note: this is about 220 Mb.  After uncompressing, the data is contained in a directory `aclImdb` with sub-directories `train` and `test`. In the following, replace the piece of the path that leads to the directory in which you have unpacked the data:

In [2]:
data_path = r'C:\Users\KennethElong(KEEL)\Downloads\aclImdb_v1\aclImdb\train'

# Load the training data
reviews_train = load_files(data_path, categories=['neg', 'pos'])

In [3]:
print(reviews_train.target_names)
revidx = 15
print(reviews_train.data[revidx])
print(reviews_train.target[revidx])

['neg', 'pos']
b'"Show People" is an absolutely delightful silent directed by King Vidor and starring Marion Davies and Billy Haines. What gems both of them are in this charming comedy about a young girl, Peggy Pepper, whose acting is the talk of Savannah trying to make it on the big screen. Though she\'s a success in comedy, what she wants to do is make "art" so she moves up to High Arts Studio. Soon she becomes Patricia Pepoire and is too good for the likes of her friend Billy.<br /><br />Many stars of the silent era have cameos in "Show People," including Davies herself without the curly hair and makeup. I\'m sure when people saw the film in 1928, they recognized everyone who appeared in the elaborate lunch scene; sadly, nowadays, it\'s not the case, even for film buffs. In one part of the film, however, she does meet Charlie Chaplin; in another, author Elinor Glyn is pointed out to her, and Vidor himself has a cameo at the end of the film. Other stars who pop up in "Show People" ar

First, we create a dictionary that maps tokens appearing in the data to indices:

In [4]:
dictionary=CountVectorizer(min_df=0.0005, max_df=0.5).fit(reviews_train.data)

In [7]:
print("The dictionary contains {} entries".format(len(dictionary.vocabulary_)))
wrd='horrible'
#wrd='is'

print("The index of the word '{}' in the dictionary is {}".format(wrd,dictionary.vocabulary_[wrd]))
widx=708
print("The word at index {} is '{}'".format(widx, dictionary.get_feature_names_out()[widx]))

The dictionary contains 15862 entries
The index of the word 'horrible' in the dictionary is 6953
The word at index 708 is 'an'


The dictionary construction with `CountVectorizer` does not include stemming:

In [9]:
print(dictionary.get_feature_names_out()[6951:6956])

['horrendous' 'horrendously' 'horrible' 'horribly' 'horrid']


Now transform the data into feature vectors:

In [10]:
reviews_train_vec=dictionary.transform(reviews_train.data)

print("The type of 'reviews_train_vec' is {} with {} rows and {} columns".format(type(reviews_train_vec),reviews_train_vec.shape[0],reviews_train_vec.shape[1]))


The type of 'reviews_train_vec' is <class 'scipy.sparse._csr.csr_matrix'> with 25000 rows and 15862 columns


The sparse matrix structure becomes visible, when we print the first 1000 entries of the row for the review at index `revidx`:

In [11]:
print(reviews_train_vec[revidx,0:1000])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 11 stored elements and shape (1, 1000)>
  Coords	Values
  (0, 44)	1
  (0, 262)	1
  (0, 271)	1
  (0, 354)	1
  (0, 430)	1
  (0, 640)	1
  (0, 708)	3
  (0, 788)	1
  (0, 857)	1
  (0, 972)	1
  (0, 989)	1


The missing 0s are still 'there':

In [12]:
print(reviews_train_vec[revidx,400])

0


We also construct a tf-idf representation of the reviews:

In [13]:
tfidf_transformer_train=TfidfTransformer().fit(reviews_train_vec)
reviews_train_tfidf_vec=tfidf_transformer_train.transform(reviews_train_vec)
print(reviews_train_tfidf_vec[revidx,0:1000])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 11 stored elements and shape (1, 1000)>
  Coords	Values
  (0, 44)	0.11583415233723711
  (0, 262)	0.02736708009501029
  (0, 271)	0.05703878107352525
  (0, 354)	0.037194533735317274
  (0, 430)	0.09538182464530867
  (0, 640)	0.034471169026461794
  (0, 708)	0.07543383127629115
  (0, 788)	0.04295469192449846
  (0, 857)	0.07717698456902591
  (0, 972)	0.06196111008037628
  (0, 989)	0.0834549762184734


### Using the feature vector for prediction

Learning a Naive Bayes model:

In [14]:
mnb=MultinomialNB().fit(reviews_train_vec,reviews_train.target)

Loading and transforming the test data. The dictionary used for the test data is the one constructed from the training data! Also the transformation with the idf values is done using the TfidfTransformer constructed from the training data.

In [15]:
reviews_test=load_files(r'C:\Users\KennethElong(KEEL)\Downloads\aclImdb_v1\aclImdb\train',categories=['neg','pos'])
reviews_test_vec=dictionary.transform(reviews_test.data)
reviews_test_tfidf_vec = tfidf_transformer_train.transform(reviews_test_vec)

Applying the learned model to the test data:

In [19]:
predictions_train=mnb.predict(reviews_train_vec)
predictions_test=mnb.predict(reviews_test_vec)

print("Accuracy on test: \n {}\n".format(accuracy_score(reviews_test.target,predictions_test)))
print("Accuracy on train: \n {}\n".format(accuracy_score(reviews_train.target,predictions_train)))



Accuracy on test: 
 0.87324

Accuracy on train: 
 0.87324



We can find out about what words are most indicative of postive/negative reviews, by looking at the parameters of the model.

The attribute `feature_log_prob_` returns the log probabilities of the different features (words) under the two classes. By taking the difference for the two classes, we get a measure for how much a word discriminates between the two classes:

In [20]:
log_prob_diff=mnb.feature_log_prob_[0,:]-mnb.feature_log_prob_[1,:]

We can use `np.argsort` to obtain the indices of the values in log_prob_diff in increasing order:

In [21]:
sorted_idxs=np.argsort(log_prob_diff)
print(sorted_idxs[0:30])

[ 4636   814  4116  6443  6210 15802 10285 15226  1626  6127  1851  3723
  6576 10163  9993 14676 12154  1200  8898  3610  1609  7923 14593  6806
 12647  8048  9862 10302  6439  5534]


... and retrieve the words corresponding to these indices:

In [22]:
numfeats=20
print("The {} words most discriminating for positive reviews are:\n".format(numfeats))
for i in sorted_idxs[0:numfeats]:
    print(dictionary.get_feature_names_out()[i])
print("\n")    
print("The {} words most discriminating for negative reviews are:\n".format(numfeats))
for i in sorted_idxs[len(sorted_idxs)-numfeats:len(sorted_idxs)]:
    print(dictionary.get_feature_names_out()[i])    

The 20 words most discriminating for positive reviews are:

edie
antwone
din
gunga
goldsworthy
yokai
paulie
visconti
blandings
gino
brashear
deathtrap
harilal
panahi
ossessione
tsui
sabu
aweigh
mcintire
daisies


The 20 words most discriminating for negative reviews are:

weisz
wayans
dyer
dahmer
rosanna
dunaway
savini
beowulf
seagal
thunderbirds
manos
shaq
btk
saif
kareena
hobgoblins
tashan
slater
uwe
boll


This indicates that we are overfitting due to very rare words. We can inspect instead words that are a little bit away from the extreme ends of the sorted log_prob_diff vector: 

In [24]:
numfeats=20
offset = 500
print("Words with 'positive rank' between {} and {}:\n".format(offset,offset+numfeats))
for i in sorted_idxs[offset:offset+numfeats]:
    print(dictionary.get_feature_names_out()[i])
print("\n")    
print("Words with 'positive rank' between {} and {}:\n".format(len(sorted_idxs)-numfeats-offset,len(sorted_idxs)-offset))
for i in sorted_idxs[len(sorted_idxs)-numfeats-offset:len(sorted_idxs)-offset]:
    print(dictionary.get_feature_names_out()[i])    

Words with 'positive rank' between 500 and 520:

rainer
liu
einstein
sergeants
conductor
amicus
companionship
seamlessly
demme
trading
corinne
swordplay
cheryl
moriarty
brilliantly
iran
shanghai
mesmerizing
favorites
sullivan


Words with 'positive rank' between 15342 and 15362:

aimlessly
forehead
humourless
countdown
inexcusable
drawl
walston
horribly
horrendous
rambo
lackluster
hulk
dafoe
rehash
bother
simpson
rubber
cringing
costs
dumb


We try a neural network classifier on the tf-idf transformed data next. We use a fairly strong regularization with `alpha=0.5` to compensate for the strong overfitting opportunities in this dataset.

In [25]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(alpha=0.5).fit(reviews_train_tfidf_vec,reviews_train.target)

While this is runnig, we can take a little break!

In [26]:
predictions_train=mlp.predict(reviews_train_tfidf_vec)
predictions_test=mlp.predict(reviews_test_tfidf_vec)


print("Accuracy on test: \n {}\n".format(accuracy_score(reviews_test.target,predictions_test)))
print("Accuracy on train: \n {}\n".format(accuracy_score(reviews_train.target,predictions_train)))

Accuracy on test: 
 0.92484

Accuracy on train: 
 0.92484



We find the reviews that are evaluated as most positive (negative) by the MLP:

In [27]:
most_positive=np.argsort( mlp.predict_proba(reviews_test_tfidf_vec)[:,0] )
print(most_positive)

[ 3770  7586 19599 ...  2667 23628 14849]


In [28]:
numrevs = 3

print("The {} reviews ranked as most positive are:\n".format(numrevs))
for i in most_positive[0:numrevs]:
    print(reviews_test.data[i])
    print("\n\n")
 
print("The {} reviews ranked as most negative are:\n".format(numrevs))
for i in most_positive[len(most_positive)-numrevs:len(most_positive)]:
    print(reviews_test.data[i])
    print("\n\n")
 

The 3 reviews ranked as most positive are:

b"If you have not seen this excellent movie about life in the 90s (in L.A.) then you've missed a special treat. This is one of the most amazingly and most powerful movies ever made about life for Americans in the 90s and it even carries over into today's world in which we live in. It covers everything from raising a child, prejudice (more than one way),love, adultery, empty nest syndrome, selfishness, etc..and the list goes on. This story builds up to an ultimate climax and then when nothing else matters it always goes back to love with friends and family and love of life. It helps us dig deep within ourselves and to make us search for what we want out of life. Makes us ask questions of ourselves. Have we done enough for others, are we like this, etc.??? Sit back and enjoy a wonderfully done and emotional movie that I'm sure others will enjoy for a lifetime.<br /><br />Take note of Mary Mcdonnell, Kevin Kline and Danny Glover's wonderful perf

### Part 2: LDA

Since LDA requires some rather time-consuming computations, we first create a smaller dictionary than we had before:

In [ ]:
dictionary=CountVectorizer(min_df=0.01, max_df=0.15).fit(reviews_train.data)
print("The dictionary contains {} entries".format(len(dictionary.vocabulary_)))
reviews_train_vec=dictionary.transform(reviews_train.data)
reviews_train_tfidf_vec=TfidfTransformer().fit_transform(reviews_train_vec)

The dictionary contains 1665 entries


We now fit an LDA model with `ntopics` latent topics. The LDA learner has a parameter `max_iter` that determines how many iterations over the whole dataset are performed. A *perplexity* score measures how well the learned model explains/fits the data. When time permits, one can see how the perplexity score improves when one allows more iterations in the learning process.

In [ ]:
ntopics=20

maxiters = np.array([5,10])

for m in maxiters:
    start=time.time()
    lda = LatentDirichletAllocation(n_components=ntopics,learning_method='online',max_iter=m).fit(reviews_train_vec)
    end=time.time()
    print("Time: {}s".format(end-start))
    print("Iterations performed: {}".format(lda.n_iter_))
    print("Perplexity score: {}".format(lda.perplexity(reviews_train_vec)))

Time: 36.290708780288696s
Iterations performed: 5
Perplexity score: 1389.1235697432812
Time: 65.17953634262085s
Iterations performed: 10
Perplexity score: 1392.8813228559225


The attribute `components_` contains the word probabilities for the latent topics (not strictly probabilities, because they do not sum to 1 over all words):

In [ ]:
print(lda.components_.shape)
f=30
print("The first {} words have the following 'probabilities':\n {}\n".format(f,lda.components_[0,0:f]))

(20, 1665)
The first 30 words have the following 'probabilities':
 [7.10779694e+02 5.00000011e-02 1.29698903e+02 2.84837450e+02
 5.00000015e-02 1.79242398e+02 5.00000020e-02 5.00000018e-02
 5.00000007e-02 5.00000009e-02 5.00000034e-02 5.00000017e-02
 5.00000011e-02 5.00000014e-02 5.00000015e-02 8.65089105e+01
 5.00000011e-02 2.94644677e+02 5.00000006e-02 5.00000009e-02
 5.00000010e-02 5.00000012e-02 5.00000009e-02 4.58518071e+01
 6.48061723e+01 5.46702109e+01 5.00405434e-02 1.46751627e+02
 5.00000025e-02 5.00000009e-02]



We next investigate which words have the highest probabilities under the different topics:

In [ ]:
widx_ranks=np.argsort(lda.components_,axis=1)

numwords=20
for i in np.arange(ntopics):
    print("Topic {}\n".format(i))
    for j in np.arange(numwords):
        print(dictionary.get_feature_names()[widx_ranks[i,len(dictionary.vocabulary_)-j-1]])
    print("\n")

Topic 0

years
wonderful
mr
excellent
am
saw
again
always
loved
amazing
michael
thought
job
parents
beautiful
seeing
enjoyed
favorite
every
10


Topic 1

action
horror
effects
genre
special
lee
fans
dr
sequences
de
atmosphere
battle
gore
fight
evil
dark
may
however
style
elements


Topic 2

world
children
earth
animation
japanese
voice
our
beautiful
fantasy
spirit
animated
child
eyes
animals
power
giant
beauty
dog
live
images


Topic 3

us
american
human
true
without
understand
real
perhaps
nature
stories
british
history
hollywood
able
emotional
important
based
self
viewer
whether


Topic 4

funny
comedy
fun
humor
laugh
jokes
10
hilarious
cast
comic
moments
entertaining
perfect
filled
laughs
christmas
actors
highly
enjoy
comedies


Topic 5

years
music
dvd
novel
song
now
dance
remember
later
version
saw
star
last
released
release
few
since
title
god
night


Topic 6

killer
police
murder
wife
gets
david
death
blood
killed
night
cop
crime
80
mystery
ending
prison
slasher
detective
killin

Next, we investigate the topic distributions assigned to the different reviews. These have first to be computed using the `transform` method:

In [ ]:
topicvecs = lda.transform(reviews_train_vec)

print(topicvecs.shape)
print(topicvecs[0:3,:])

(25000, 20)
[[0.00131579 0.43100785 0.00131579 0.00131579 0.00131579 0.00131579
  0.00131579 0.00131579 0.00131579 0.05414566 0.00131579 0.00131579
  0.08458983 0.00131579 0.20784751 0.00131579 0.00131579 0.00131579
  0.10238772 0.10160037]
 [0.00111111 0.00111111 0.49372067 0.00111111 0.00111111 0.00111111
  0.00111111 0.00111111 0.00111111 0.00111111 0.03184332 0.00111111
  0.15261789 0.00111111 0.00111111 0.00111111 0.00111111 0.00111111
  0.00111111 0.30404034]
 [0.00238095 0.00238095 0.00238095 0.00238095 0.2980676  0.00238095
  0.00238095 0.00238095 0.00238095 0.00238095 0.00238095 0.00238095
  0.08461675 0.00238095 0.20732149 0.00238095 0.00238095 0.00238095
  0.00238095 0.37189893]]


We can use the topic distributions as feature vectors to cluster the movies. The number of clusters we here choose can be quite different from the number of topics in the LDA model.

In [ ]:
numclus=40
docclus=KMeans(n_clusters=numclus,n_init=1).fit(topicvecs)

print("Cluster sizes:\n")
for i in np.arange(numclus):
    print("{}: {}".format(i,len(np.where(docclus.labels_==i)[0])))

print("\n")
print("Silhouette score: {}".format(silhouette_score(topicvecs,docclus.labels_)))    
    

Cluster sizes:

0: 604
1: 551
2: 481
3: 257
4: 623
5: 333
6: 607
7: 382
8: 1000
9: 482
10: 1007
11: 578
12: 954
13: 486
14: 427
15: 458
16: 604
17: 479
18: 446
19: 911
20: 1063
21: 447
22: 838
23: 583
24: 609
25: 720
26: 837
27: 1006
28: 450
29: 419
30: 954
31: 321
32: 487
33: 635
34: 590
35: 486
36: 670
37: 477
38: 954
39: 784


Silhouette score: 0.08037105092085178


The quite uniform size distribution and the low Silhouette score indicate that we have not found a very clear or natural clustering. However, we can still use the cluster centers, and the reviews closest to the cluster centers as  representatives for different types of reviews.

As before for the 'Faces' dataset, we use the `transform` method to compute a distance matrix of the datapoints to the cluster centers.

In [ ]:
distances=docclus.transform(topicvecs)
centrals=np.argsort(distances,axis=0)

print(topicvecs.shape)
print(distances.shape)
print(centrals.shape)

(25000, 20)
(25000, 40)
(25000, 40)


Now we can inspect the reviews that are closest to the center of a given cluster. The following first prints the topic distribution vectors of the `norevs` reviews that are closest to the cluster center as columns in a matrix. The first column of the matrix just contains the indices of the topics. Then the actual reviews are printed.

In [ ]:
# The index of the cluster we want to investigate
clusterno=6
# The number of reviews closest to the cluster center we want to inspect
norevs=3

topicparams = np.zeros([ntopics,norevs+1])
topicparams[:,0]=np.arange(ntopics)

for i in np.arange(norevs):
    topicparams[:,i+1]=topicvecs[centrals[i,clusterno],:]
 

print(np.array_str(topicparams, precision=2))
print("\n")

#for j in np.arange(topicvecs.shape[1]):
#    print("{}: {:.3}".format(j,topicvecs[centrals[i,clusterno],j]))
        
for i in np.arange(norevs):    
    print(reviews_train.data[centrals[i,clusterno]])

    print("\n")

[[0.00e+00 2.25e-04 3.03e-04 6.79e-02]
 [1.00e+00 6.96e-02 6.70e-02 3.03e-02]
 [2.00e+00 5.91e-02 4.51e-02 6.09e-02]
 [3.00e+00 6.37e-02 6.22e-02 1.04e-01]
 [4.00e+00 2.25e-04 3.03e-04 6.35e-02]
 [5.00e+00 6.04e-02 6.94e-02 1.84e-04]
 [6.00e+00 1.91e-02 1.02e-02 1.84e-04]
 [7.00e+00 3.51e-02 1.77e-02 2.97e-02]
 [8.00e+00 2.25e-04 3.03e-04 9.73e-03]
 [9.00e+00 2.25e-04 3.59e-02 1.39e-02]
 [1.00e+01 1.41e-02 9.26e-03 1.87e-02]
 [1.10e+01 1.02e-01 3.03e-04 5.51e-02]
 [1.20e+01 2.25e-04 3.03e-04 1.84e-04]
 [1.30e+01 5.58e-02 9.43e-02 1.84e-04]
 [1.40e+01 3.47e-01 4.51e-01 3.31e-01]
 [1.50e+01 2.25e-04 1.46e-02 3.15e-02]
 [1.60e+01 1.44e-02 6.91e-02 1.18e-02]
 [1.70e+01 4.07e-02 3.50e-02 6.56e-02]
 [1.80e+01 5.03e-02 1.71e-02 4.68e-02]
 [1.90e+01 6.74e-02 3.03e-04 5.94e-02]]


b'Film can be a looking glass to see the world in a new light. Good Night and Good Luck, for instance, offered parallels to modern judgement-without-evidence and encroachments on freedom. It is easier to examine a mor

Finally, we can look which reviews are most strongly connected to any specific topic

In [ ]:
topicvecs.shape
toprevs_by_topic=np.argsort(topicvecs,axis=0)

In [ ]:
# The index of the topic we want to investigate
topicno=8
# The number of reviews most highly associated with the topic that we want to see
norevs=4

l=len(toprevs_by_topic)-1

for i in np.arange(norevs):    
    print(reviews_train.data[toprevs_by_topic[l-i,topicno]])

    print("\n")

b'For those of you unfamiliar with Jimmy Stewart, this is one of his "lesser" films from later in his career. And, while it isn\'t a great film compared to many of his other pictures, it isn\'t bad and is a decent time-passer--but not much more.<br /><br />Kim Novak is a witch in New York City and for some inexplicable reason, she decides to cast a spell on poor Jimmy to make him fall in love with her. Over time, the cold and detached Ms. Novak also begins to fall in love with Stewart--and apparently in the witch\'s rule book, this is a definite NO, NO!! <br /><br />The film is odd in its sensibilities about the witches. They are neither the baby-sacrificing nor the all-powerful variety. Most of their magic is pretty limited and pointless (such as Jack Lemmon using his powers to turn off street lamps). And, very oddly, the witches all seem to be bohemians who hang out in hip bars where you might find people wearing berets and listening to crappy jazz. Considering what I think of jazz, 